# Load Models & Modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Future print function
from __future__ import print_function

# Load modules/models from EVA course
!git clone https://github.com/sujitojha1/ERA1 -q --quiet
!mv ERA1/Session13/* .
!rm -rf ERA1

print("Loaded Modules and Models")

# Installing latest Albumentation library
!pip install -U git+https://github.com/albu/albumentations -q --quiet
!pip install torch-lr-finder -q --quiet
!pip install grad-cam -q --quiet
!pip install --quiet "pandas" "ipython[notebook]" "seaborn" "torchmetrics" "lightning-bolts" "lightning" "kaggle"

# For inline matplotlib plotting
%matplotlib inline

Loaded Modules and Models


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"kanha83","key":"xxx"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [15]:
!kaggle datasets download -d aladdinpersson/pascal-voc-dataset-used-in-yolov3-video

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [14]:
!unzip -qq pascal-voc-dataset-used-in-yolov3-video.zip
!rm -rf pascal-voc-dataset-used-in-yolov3-video.zip

unzip:  cannot find or open pascal-voc-dataset-used-in-yolov3-video.zip, pascal-voc-dataset-used-in-yolov3-video.zip.zip or pascal-voc-dataset-used-in-yolov3-video.zip.ZIP.


# Loading train and test data with transforms and loader functions

In [1]:
!git clone 'https://github.com/shilpiprd/ERA-Solution.git'
%cd ERA-Solution/Sess13\ Solution/base_S13/

print("Downloading libraries: ")
! pip install pytorch-lightning -q --quiet
! pip install torch-lr-finder -q --quiet
! pip install torchvision -q --quiet

print("Modules and Models loaded: ")
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Now you can use kaggle commands to download datasets


Cloning into 'ERA-Solution'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 372 (delta 29), reused 145 (delta 24), pack-reused 222
Receiving objects: 100% (372/372), 189.10 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Updating files: 100% (286/286), done.
/content/ERA-Solution/Sess13 Solution/base_S13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 17.0 MB/s eta 0:00:00
Modules and Models loaded: 


In [2]:
!kaggle datasets download -d aladdinpersson/pascal-voc-yolo-works-with-albumentations

100% 4.29G/4.31G [00:45<00:00, 169MB/s]
100% 4.31G/4.31G [00:45<00:00, 101MB/s]


In [3]:
!unzip -qq pascal-voc-yolo-works-with-albumentations.zip

In [4]:
! ls

config.py				 image.png    model.py
dataloader.ipynb			 kaggle.json  PASCAL_VOC
dataset_org.py				 loss_1_1.py  pascal-voc-yolo-works-with-albumentations.zip
dataset.py				 loss.py      pl_detection.py
ERA1_S13_YOLOv3_Pytorch_lightning.ipynb  main2.ipynb  train.py
Exp3.ipynb				 main.ipynb   utils.py


In [5]:
import warnings
import config
warnings.filterwarnings("ignore")

In [8]:
# from lightning import LightningModule, Trainer, seed_everything
# from lightning.pytorch.callbacks import LearningRateMonitor
# from lightning.pytorch.callbacks.progress import TQDMProgressBar
# from lightning.pytorch.loggers import CSVLogger
# from torch.optim.lr_scheduler import OneCycleLR
# from torch.optim.swa_utils import AveragedModel, update_bn
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch.optim.lr_scheduler import OneCycleLR

from pl_detection import *

model = LitYOLOv3()

trainer = Trainer(
    max_epochs=config.NUM_EPOCHS * 2 // 5,
    precision="16-mixed",
    accelerator="auto",
    # devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    devices = "auto",
    logger=CSVLogger(save_dir="logs/"),
    callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)],
)

trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type   | Params
---------------------------------
0 | model | YOLOv3 | 61.6 M
---------------------------------
61.6 M    Trainable params
0         Non-trainable params
61.6 M    Total params
246.506   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from lightning.pytorch.tuner import Tuner

# trainer_temp = Trainer(
#     max_epochs=200,
#     accelerator="auto",
#     devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
# )

# # Create a Tuner
# tuner = Tuner(trainer_temp)

# # finds learning rate automatically
# # sets hparams.lr or hparams.learning_rate to that learning rate
# tuner.lr_find(model)

In [ ]:
torch.save(model.model.state_dict(), "exp8_custom_yolo_v3.pt")